Copyright 2024 Zhixuan Hu

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import numpy as np
import pandas as pd
import os, re, sys, json, time
sys.path.append(os.getcwd() + '\\..\\')

In [17]:
pattern = re.compile(r".*\.csv")
files = os.listdir('../../dataset/')
files = [file for file in files if pattern.match(file)]
files

['chatgpt_HC3_all.csv',
 'chatgpt_HC3_all.csv-2024-06-04-21-04-24.csv',
 'chatgpt_HC3_all.csv-2024-06-05-12-05-06.csv',
 'chatgpt_HC3_all.csv-2024-06-05-12-39-20.csv',
 'gpt_0125_00.csv',
 'human_00.csv',
 'human_01_01.csv',
 'human_01_02.csv',
 'human_02.csv',
 'human_cook_01.csv',
 'human_cook_02.csv',
 'human_HC3_all.csv',
 'human_HC3_all.csv-2024-06-04-21-02-55.csv',
 'human_HC3_all.csv-2024-06-05-11-36-01.csv',
 'human_modern_01.csv',
 'human_modern_02.csv',
 'human_travel_01.csv',
 'human_travel_02.csv',
 'human_travel_03.csv',
 'questions_00.csv',
 'questions_01_01.csv',
 'questions_01_02.csv',
 'questions_02.csv',
 'questions_cook_01.csv',
 'questions_cook_02.csv',
 'questions_HC3_all.csv',
 'questions_modern_01.csv',
 'questions_modern_02.csv',
 'questions_travel_01.csv',
 'questions_travel_02.csv',
 'questions_travel_03.csv',
 'test_gpt_0125.csv',
 'test_gpt_0125.csv-2024-06-04-21-05-58.csv',
 'test_gpt_0125.csv-2024-06-05-10-49-57.csv',
 'test_human.csv',
 'test_human.csv-20

In [25]:
file = '../../dataset/test_gpt_0125.csv'
df = pd.read_csv(file)
df.head(1)

,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,question_id,answer_id,text,text-embedding-3-small,"sentences_length[mean1,cv1,mean2,cv2]","words_richness[ttr, yules_k, d_index]","emotion_Dou[eps, eiv]"
0,0,0,0,0,0,0.0,454,1,深度学习是一种机器学习的方法，它通过模拟人类神经网络的结构和功能来实现对数据的学习和预测。在...,"[-0.03873550891876221, -0.03209991380572319, 0...","[41.611111111111114, 0.25685931847790017, 14.3...","[0.33986928104575165, 235.04729899706192, 0.08...","[-0.99242573436361, 0.013853045618768956]"


In [26]:
model = 'emotion_Dou[eps, eiv]'
# 检查是否存在 'as' 列
if model not in df.columns:
    # 如果不存在，则创建一个空的字符串列
    df[model] = ''
df[model] = df[model].astype(str)

In [27]:
df_need = df[('"nan"' == df[model]) | (df[model]=='')]
# 防止过长超出token限制
tol = 1e20
mask = df_need['text'].str.len() > tol
df_need.loc[mask, 'text'] = df_need.loc[mask, 'text'].str.slice(0, tol) #截断
df_need_list = df_need['text'].to_list()
df_need.head(1)

,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,question_id,answer_id,text,text-embedding-3-small,"sentences_length[mean1,cv1,mean2,cv2]","words_richness[ttr, yules_k, d_index]","emotion_Dou[eps, eiv]"
0,0,0,0,0,0,0.0,454,1,深度学习是一种机器学习的方法，它通过模拟人类神经网络的结构和功能来实现对数据的学习和预测。在...,"[-0.03873550891876221, -0.03209991380572319, 0...","[41.611111111111114, 0.25685931847790017, 14.3...","[0.33986928104575165, 235.04729899706192, 0.08...","[-0.99242573436361, 0.013853045618768956]"


In [6]:
# 单进程
# from emotion_Dou import main
# df_need[model] = df_need['text'].apply(lambda row: json.dumps(main(row)))

In [28]:
# 多进程
import json
from emotion_Dou import main
from joblib import Parallel, delayed

def process_row(row):
    return json.dumps(main(row))

# 使用 joblib 进行并行计算
results = Parallel(n_jobs=-1)(delayed(process_row)(row) for row in df_need['text'])

# 将计算结果赋值回 DataFrame
df_need[model] = results


In [29]:
indexer = df_need.index
df.loc[indexer] = df_need
df[df.columns[8:]]

,text,text-embedding-3-small,"sentences_length[mean1,cv1,mean2,cv2]","words_richness[ttr, yules_k, d_index]","emotion_Dou[eps, eiv]"
0,深度学习是一种机器学习的方法，它通过模拟人类神经网络的结构和功能来实现对数据的学习和预测。在...,"[-0.03873550891876221, -0.03209991380572319, 0...","[41.611111111111114, 0.25685931847790017, 14.3...","[0.33986928104575165, 235.04729899706192, 0.08...","[-0.99242573436361, 0.013853045618768956]"
1,深度学习作为一种机器学习方法，其本质是通过神经网络来实现对数据的学习和模式识别。尽管深度学习...,"[-0.03281135484576225, -0.04714532196521759, 0...","[43.642857142857146, 0.3040535969520589, 16.36...","[0.4383561643835616, 172.49014824544943, 0.137...","[-0.9971031135266845, 0.003428978282686927]"
2,深度学习并没有脱离数学，相反，数学在深度学习中扮演着非常重要的角色。深度学习是建立在数学基础...,"[-0.05262744054198265, -0.041652824729681015, ...","[47.2, 0.22277015979821888, 13.176470588235293...","[0.5179856115107914, 220.4854821179028, 0.2460...","[-0.9999971667189304, 2.2681937399698345e-06]"
3,深度学习是一门需要深厚理论基础和丰富实践经验的学科，纯靠造假是无法取得真正的科研成果和发表高...,"[0.05532760173082352, -0.00803462602198124, 0....","[43.36363636363637, 0.28333703678015437, 15.26...","[0.5083056478405316, 160.0423836381497, 0.1719...","[-0.9096644141027803, 0.12414851826658714]"
4,不应该仅仅依靠造假来发表论文。虽然有些人可能会通过欺骗和伪造数据来发表论文，但这种行为是极其...,"[0.07378417253494263, -0.026973411440849304, 0...","[34.5, 0.4222841241835787, 16.75, 0.2248734204...","[0.6547619047619048, 167.23356009070295, 0.415...","[-0.7869503595847821, 0.07258366418451918]"
...,...,...,...,...,...
319,鸿蒙系统是华为公司自行研发的操作系统，它与安卓系统有所不同，虽然两者都是基于Linux内核开...,"[0.009668500162661076, 0.00952873844653368, 0....","[50.18181818181818, 0.3370928507134205, 14.638...","[0.4646153846153846, 150.53254437869822, 0.169...","[-0.7812304893657067, 0.07706697231436901]"
320,鸿蒙系统和安卓系统是两个不同的操作系统。鸿蒙系统是由华为公司自主研发的操作系统，旨在为物联网...,"[-0.012859110720455647, 0.021412188187241554, ...","[35.333333333333336, 0.43621237817724123, 17.1...","[0.711864406779661, 143.63688595231255, 0.5412...","[-0.91433732588865, 0.08247039458465749]"
321,关于黑人需要混几代血才能接近白人肤色这个问题，首先我认为肤色并不是衡量一个人价值和身份的标准...,"[0.06337092816829681, -0.013654569163918495, 0...","[36.1764705882353, 0.35554398817855076, 13.044...","[0.34146341463414637, 207.49553837001784, 0.10...","[-0.7865466427458667, 0.15192894312026786]"
322,种族和肤色并不代表一个人的性格和人性，每个人都有自己独特的特质和价值，不应该用肤色来衡量一个...,"[0.05830851197242737, 0.0020467150025069714, 0...","[32.4, 0.38776046356887434, 10.928571428571429...","[0.5922330097087378, 201.71552455462344, 0.324...","[-0.6353338957089568, 0.02992310353455195]"


In [30]:
time_str = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())
os.rename(file, '../../dataset/history/' + file + '-' + time_str + '.csv')
df.to_csv(file, encoding='utf-8-sig')